   ## **Case - Modelo preditivo para aprovação de crédito**
Modelos de score de crédito calculam a probabilidade de inadimplência e são uma das principais ferramentas utilizadas por diversas empresas para aprovar ou negar um crédito.
O objetivo deste desafio é criar um modelo preditivo calculando a probabilidade de inadimplência de cada novo pedido de crédito.
Dados:

### Cada linha representa um cliente e as colunas representam os dados (informações) desses clientes.
### A variável resposta é a coluna inadimplente, que indica se o tomador veio a se tornar inadimplente(1) ou não(0).
### As variáveis da base de dados são descritas abaixo:



    ● idade: A idade do cliente..
    
    ● numero_de_dependentes: O número de pessoas dependentes do cliente.
    
    ● salario_mensal: Salário mensal do cliente.
    
    ● numero_emprestimos_imobiliarios: Quantidade de empréstimos imobiliários que o cliente possui em aberto.

    ● numero_vezes_passou_90_dias: Número de vezes que o tomador passou mais de 90 dias em atraso.

    ● util_linhas_inseguras: Quanto que o cliente está usando, relativamente ao limite dele, de linhas de crédito que não são seguradas por qualquer bem do tomador e.g: imoveis, carros etc.

    ● vezes_passou_de_30_59_dias: Número de vezes que o cliente atrasou, entre 30 e 59 dias, o pagamento de um empréstimo.

    ● razao_debito: Razão entre as dívidas e o patrimônio do tomador. razão débito = Dividas/Patrimônio

    ● numero_linhas_crdto_aberto: Número de empréstimos em aberto pelo cliente.
    
    ● numero_de_vezes_que_passou_60_89_dias: Número de vezes que o cliente atrasou, entre 60 e 89 dias, o pagamento de um empréstimo.


#### Obs:​ Estes dados foram retirados de terceiros, portanto é possível que existam incoerências, o que é perfeitamente comum em dados reais.
### **Objetivo:**
    Construir qualquer modelo preditivo utilizando o arquivo treino.csv.
    
    Utilize este modelo para gerar as previsões na base teste.csv, inserindo uma nova coluna na tabela de dados do arquivo teste.csv que contenha as previsões e nomeie esta coluna com o nome "inadimplente".

# **Planejamento da solução**

## Input


### O problema de negocio

1. Selecionar os melhores clientes com base no historico e comportamento de credito dos clientes, classificar os novos como sendo possivel liberação de credito ou não. 


### Output

1. Classificação de novos clientes para terem solicitação de credito aprovada ou não

## 0.0. **Imports**

In [313]:
import math
import numpy  as np
import pandas as pd
import random
import pickle
import warnings
import inflection
import seaborn as sns
import xgboost as xgb

from scipy                 import stats  as ss
from boruta                import BorutaPy
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML


from sklearn.metrics       import mean_absolute_error, mean_squared_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder

### 0.1. Helper Function

### **0.2. Load dataset**

In [387]:
df_raw_teste = pd.read_csv("data/teste.csv")
df_raw_treino = pd.read_csv("data/treino.csv")

In [388]:
df_raw_teste.head()

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0
1,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0
2,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0
3,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0
4,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0


In [389]:
df_raw_treino.head()

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


## **1.0. Data descripitions**

In [390]:
# Todo o conhecimento dos dados sera feito nos dados de treino
df1 = df_raw_treino.copy()
df_teste1 = df_raw_teste

### **1.1. Rename columns**

In [391]:
df1.columns.to_list()

['inadimplente',
 'util_linhas_inseguras',
 'idade',
 'vezes_passou_de_30_59_dias',
 'razao_debito',
 'salario_mensal',
 'numero_linhas_crdto_aberto',
 'numero_vezes_passou_90_dias',
 'numero_emprestimos_imobiliarios',
 'numero_de_vezes_que_passou_60_89_dias',
 'numero_de_dependentes']

In [392]:
cols_news = ['inadimplente', 'linhas_inseguras', 'idade', 'qtd_passou_de_30_59_dias', 'razao_debito',
             'salario_mensal', 'qtd_crdto_aberto', 'qtd_passou_90_dias', 'qtd_emp_imobiliarios',
             'qtd_passou_60_89_dias', 'qtd_dependentes']
df1.columns = cols_news


cols_news_teste = ['linhas_inseguras', 'idade', 'qtd_passou_de_30_59_dias', 'razao_debito',
             'salario_mensal', 'qtd_crdto_aberto', 'qtd_passou_90_dias', 'qtd_emp_imobiliarios',
             'qtd_passou_60_89_dias', 'qtd_dependentes']

df_teste1.columns = cols_news_teste


In [393]:
df1.columns.to_list()

['inadimplente',
 'linhas_inseguras',
 'idade',
 'qtd_passou_de_30_59_dias',
 'razao_debito',
 'salario_mensal',
 'qtd_crdto_aberto',
 'qtd_passou_90_dias',
 'qtd_emp_imobiliarios',
 'qtd_passou_60_89_dias',
 'qtd_dependentes']

In [394]:
df_teste1.columns.to_list()

['linhas_inseguras',
 'idade',
 'qtd_passou_de_30_59_dias',
 'razao_debito',
 'salario_mensal',
 'qtd_crdto_aberto',
 'qtd_passou_90_dias',
 'qtd_emp_imobiliarios',
 'qtd_passou_60_89_dias',
 'qtd_dependentes']

### **1.2. Data dimensions**

In [395]:
print('Number of cols Train: {}'.format(df1.shape[1]))
print('Number of rows Train: {}'.format(df1.shape[0]))
print('Number of cols Test: {}'.format(df1.shape[1]))
print('Number of rows Test: {}'.format(df1.shape[0]))

Number of cols Train: 11
Number of rows Train: 110000
Number of cols Test: 11
Number of rows Test: 110000


### **1.3. Data types**

In [396]:
df1.dtypes

inadimplente                  int64
linhas_inseguras            float64
idade                         int64
qtd_passou_de_30_59_dias      int64
razao_debito                float64
salario_mensal              float64
qtd_crdto_aberto              int64
qtd_passou_90_dias            int64
qtd_emp_imobiliarios          int64
qtd_passou_60_89_dias         int64
qtd_dependentes             float64
dtype: object

### **1.4 Check NAs**

In [397]:
df1.isna().sum()

inadimplente                    0
linhas_inseguras                0
idade                           0
qtd_passou_de_30_59_dias        0
razao_debito                    0
salario_mensal              21763
qtd_crdto_aberto                0
qtd_passou_90_dias              0
qtd_emp_imobiliarios            0
qtd_passou_60_89_dias           0
qtd_dependentes              2878
dtype: int64

#### **OBS 1:** Estou tomando como premissa no primeiro ciclo do CRIPS que a regra de negocio do sistema que contem as informações considera valores NAs para clientes que nunca atrasaram em nenhum dos periodos.
#### **OBS 2:** Estou tomando como premissa no primeiro ciclo do CRIPS que clientes que não enviaram seus respectivos salarios devem ser considerados para não perder clientes como recebendo 1 salario minimo no Brasil.
#### **OBS 3:** Estou tomando como premissa no primeiro ciclo do CRIPS que clientes com NAs em qtd_dependentes não possuem dependentes então o numero que melhor representa é 0 .

### **1.5. Replace NAs**

#### **Se as colunas qtd_passou_90_dias e qtd_passou_60_89_dias representam a quantidade de vezes que esses clientes passou X quantidade de dias em atraso, o valor que melhor representa clientes que nuca atrasaram é 0**

In [398]:
df1['qtd_passou_90_dias'] = df1['qtd_passou_90_dias'].apply( lambda x: 0 if math.isnan( x ) else x )
df1['qtd_passou_60_89_dias'] = df1['qtd_passou_60_89_dias'].apply(lambda x: 0 if math.isnan( x ) else x)
df1['salario_mensal'] = df1['salario_mensal'].apply(lambda x: 1.212 if math.isnan( x ) else x)
df1['qtd_dependentes'] = df1['qtd_dependentes'].apply(lambda x: 0 if math.isnan( x ) else x)


In [399]:
df1.isna().sum()

inadimplente                0
linhas_inseguras            0
idade                       0
qtd_passou_de_30_59_dias    0
razao_debito                0
salario_mensal              0
qtd_crdto_aberto            0
qtd_passou_90_dias          0
qtd_emp_imobiliarios        0
qtd_passou_60_89_dias       0
qtd_dependentes             0
dtype: int64

### **1.6. Change dtypes**

In [400]:
df1.dtypes

inadimplente                  int64
linhas_inseguras            float64
idade                         int64
qtd_passou_de_30_59_dias      int64
razao_debito                float64
salario_mensal              float64
qtd_crdto_aberto              int64
qtd_passou_90_dias            int64
qtd_emp_imobiliarios          int64
qtd_passou_60_89_dias         int64
qtd_dependentes             float64
dtype: object

In [401]:
df1.head()

,inadimplente,linhas_inseguras,idade,qtd_passou_de_30_59_dias,razao_debito,salario_mensal,qtd_crdto_aberto,qtd_passou_90_dias,qtd_emp_imobiliarios,qtd_passou_60_89_dias,qtd_dependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


#### OBS: A premissa é que este valor seja referente a porcentagem. 
#### Exemplo: utilizou 95,2% do total do seu limite de credito
linhas_inseguras(util_linhas_inseguras): Quanto que o cliente está usando, relativamente ao limite dele, de linhas de crédito que não são seguradas por qualquer bem do tomador e.g: imoveis, carros etc.

In [402]:
# linhas_inseguras
## Para um proximo ciclo talvez seja interessante a conversão para int

# qtd_dependentes
df1['qtd_dependentes'] = df1['qtd_dependentes'].astype( int )
df1.dtypes

inadimplente                  int64
linhas_inseguras            float64
idade                         int64
qtd_passou_de_30_59_dias      int64
razao_debito                float64
salario_mensal              float64
qtd_crdto_aberto              int64
qtd_passou_90_dias            int64
qtd_emp_imobiliarios          int64
qtd_passou_60_89_dias         int64
qtd_dependentes               int64
dtype: object

### **1.7. Descriptive Statistics**

In [403]:
# Irei verificar no proximo ciclo, foco em rodar o ML a primeira vez paravalidar dados e aprendizado

## **2.0. Feature engenering**

In [453]:
df_teste2 = df_teste1.copy()

In [454]:
df_teste2['inadimplente'] = 0

In [455]:
df_teste2.head()

,linhas_inseguras,idade,qtd_passou_de_30_59_dias,razao_debito,salario_mensal,qtd_crdto_aberto,qtd_passou_90_dias,qtd_emp_imobiliarios,qtd_passou_60_89_dias,qtd_dependentes,inadimplente
0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0,0
1,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0,0
2,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0,0
3,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0,0
4,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0,0


In [458]:

df_teste2 = df_teste2.reindex(columns=['inadimplente',
                                       'linhas_inseguras',
                                       'idade',
                                       'qtd_passou_de_30_59_dias',
                                       'razao_debito',
                                       'salario_mensal',
                                       'qtd_crdto_aberto',
                                       'qtd_passou_90_dias',
                                       'qtd_emp_imobiliarios',
                                       'qtd_passou_60_89_dias',
                                       'qtd_dependentes',
                                       'grupo_idade',
                                       'classe_financeira'])

In [459]:
df_teste2.head(10)

,inadimplente,linhas_inseguras,idade,qtd_passou_de_30_59_dias,razao_debito,salario_mensal,qtd_crdto_aberto,qtd_passou_90_dias,qtd_emp_imobiliarios,qtd_passou_60_89_dias,qtd_dependentes,grupo_idade,classe_financeira
0,0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0,NaN,NaN
1,0,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0,NaN,NaN
2,0,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0,NaN,NaN
3,0,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0,NaN,NaN
4,0,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0,NaN,NaN
5,0,0.323985,42,0,0.352151,10415.0,8,0,2,0,2.0,NaN,NaN
6,0,0.007300,63,0,0.002289,4368.0,2,0,0,0,0.0,NaN,NaN
7,0,0.000000,76,1,0.261611,1700.0,4,0,0,0,0.0,NaN,NaN
8,0,0.062280,55,0,0.406667,15658.0,14,1,3,0,0.0,NaN,NaN
9,0,0.479899,41,1,0.209903,8441.0,6,0,3,0,0.0,NaN,NaN


In [460]:
df2 = df1.copy()

In [484]:
# Teste
df_teste2['qtd_passou_90_dias'] = df_teste2['qtd_passou_90_dias'].apply( lambda x: 0 if math.isnan( x ) else x )
df_teste2['qtd_passou_60_89_dias'] = df_teste2['qtd_passou_60_89_dias'].apply(lambda x: 0 if math.isnan( x ) else x)
df_teste2['salario_mensal'] = df_teste2['salario_mensal'].apply(lambda x: 1.212 if math.isnan( x ) else x)
df_teste2['qtd_dependentes'] = df_teste2['qtd_dependentes'].apply(lambda x: 0 if math.isnan( x ) else x)

In [485]:
# Idade(jovem, adulto, idoso)
## Jovem <= 26 == 1
## Adulto => 27 && <= 58 == 2
## Idoso >= 59 == 3
df2['grupo_idade'] = df2['idade'].apply(lambda x: '1' if x <= 26 else '2' if (x >= 27) & (x <= 58) else '3')


# Classe(baixa, media, alta)
## salario classe baixa <= 2424 == 1
## salario classe media > 2424 && <= 9999 == 2
## salario classe alta > 9999 == 3
df2['classe_financeira'] = df2['salario_mensal'].apply(lambda x: '1' if x <= 2424 else '2' if (x >= 2425) & (x <= 9999) else '3')


# Dataframe de testes
df_teste2['grupo_idade'] = df_teste2['idade'].apply(lambda x: '1' if x <= 26 else '2' if (x >= 27) & (x <= 58) else '3')
df_teste2['classe_financeira'] = df_teste2['salario_mensal'].apply(lambda x: '1' if x <= 2424 else '2' if (x >= 2425) & (x <= 9999) else '3')



In [486]:
df_teste2.isna().sum()

inadimplente                0
linhas_inseguras            0
idade                       0
qtd_passou_de_30_59_dias    0
razao_debito                0
salario_mensal              0
qtd_crdto_aberto            0
qtd_passou_90_dias          0
qtd_emp_imobiliarios        0
qtd_passou_60_89_dias       0
qtd_dependentes             0
grupo_idade                 0
classe_financeira           0
dtype: int64

In [487]:
df_teste2.head()

,inadimplente,linhas_inseguras,idade,qtd_passou_de_30_59_dias,razao_debito,salario_mensal,qtd_crdto_aberto,qtd_passou_90_dias,qtd_emp_imobiliarios,qtd_passou_60_89_dias,qtd_dependentes,grupo_idade,classe_financeira
0,0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0,3,2
1,0,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0,2,1
2,0,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0,2,2
3,0,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0,2,2
4,0,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0,2,1


In [488]:
df2[['grupo_idade']].value_counts().reset_index()

,grupo_idade,0
0,2,69143
1,3,37727
2,1,3130


In [489]:
df2[['classe_financeira']].value_counts().reset_index()

,classe_financeira,0
0,2,61940
1,1,33531
2,3,14529


## **3.0. Variable filtering**

In [490]:
df3 = df2.copy()

## **4.0. Exploratory data analysis**

In [491]:
df4 = df3.copy()

## **5.0. Data preparation**

In [492]:
df5 = df4.copy()

## **6.0. Feature selection**

In [493]:
df6 = df5.copy()

### 6.1. Manual feature selection

In [494]:
# Training dataset
X_train = df6
y_train = X_train['inadimplente']


# Test dataset
X_test = df_teste2
y_test = X_test['inadimplente']

In [495]:
df_teste2.isna().sum()

inadimplente                0
linhas_inseguras            0
idade                       0
qtd_passou_de_30_59_dias    0
razao_debito                0
salario_mensal              0
qtd_crdto_aberto            0
qtd_passou_90_dias          0
qtd_emp_imobiliarios        0
qtd_passou_60_89_dias       0
qtd_dependentes             0
grupo_idade                 0
classe_financeira           0
dtype: int64

In [496]:
df6.columns.to_list()

['inadimplente',
 'linhas_inseguras',
 'idade',
 'qtd_passou_de_30_59_dias',
 'razao_debito',
 'salario_mensal',
 'qtd_crdto_aberto',
 'qtd_passou_90_dias',
 'qtd_emp_imobiliarios',
 'qtd_passou_60_89_dias',
 'qtd_dependentes',
 'grupo_idade',
 'classe_financeira']

In [497]:
cols_selected_manual = ['inadimplente',
                         'linhas_inseguras',
                         'idade',
                         'qtd_passou_de_30_59_dias',
                         'razao_debito',
                         'salario_mensal',
                         'qtd_crdto_aberto',
                         'qtd_passou_90_dias',
                         'qtd_emp_imobiliarios',
                         'qtd_passou_60_89_dias',
                         'qtd_dependentes',
                         'grupo_idade',
                         'classe_financeira']

## **7.0. Machinle learning modelling**

In [498]:
df7 = df6.copy()

In [499]:
# Realizei a seleção da forma acima pois pode existir casos em que a base de treino e teste seja a mesma, então dessa forma fica mais facil eu separar o X_train em uma periodo especifico e o restante eu utilizar para teste
x_train = X_train[cols_selected_manual]
x_test = X_test[cols_selected_manual]

In [508]:
def mean_absolute_percentage_error(y, yhat):
    return np.mean(np.abs((y - yhat) / y ))


def ml_error (model_name, y, yhat):
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    return pd.DataFrame({'Model Name': model_name,
                         'MAE': mae,
                         'MAPE': mape,
                         'RMSE': rmse}, index=[0])

### 7.1. Average Model

In [509]:
aux1 = x_test.copy()
aux1['inadimplente'] = y_test.copy()


# prediction
aux2 = aux1[['sales']].mean().reset_index().rename( columns={'inadimplente': 'predictions_inadimplente} )
aux1 = pd.merge( aux1, aux2, how='left', on='store' )
yhat_baseline = aux1['predictions']

# performance
baseline_result = ml_error( 'Average Model', np.expm1( y_test ), np.expm1( yhat_baseline ) )
baseline_result

SyntaxError: EOL while scanning string literal (4162036898.py, line 6)

### 7.2. Linear Regresion Model

In [510]:
x_train.dtypes

inadimplente                int64
linhas_inseguras            int64
idade                       int64
qtd_passou_de_30_59_dias    int64
razao_debito                int64
salario_mensal              int64
qtd_crdto_aberto            int64
qtd_passou_90_dias          int64
qtd_emp_imobiliarios        int64
qtd_passou_60_89_dias       int64
qtd_dependentes             int64
grupo_idade                 int64
classe_financeira           int64
dtype: object

In [511]:
x_train['linhas_inseguras'] = x_train['linhas_inseguras'].astype( int )
x_train['razao_debito'] = x_train['razao_debito'].astype( int )
x_train['salario_mensal'] = x_train['salario_mensal'].astype( int )
x_train['grupo_idade'] = x_train['grupo_idade'].astype( int )
x_train['classe_financeira'] = x_train['classe_financeira'].astype( int )
x_train.isna().sum()

inadimplente                0
linhas_inseguras            0
idade                       0
qtd_passou_de_30_59_dias    0
razao_debito                0
salario_mensal              0
qtd_crdto_aberto            0
qtd_passou_90_dias          0
qtd_emp_imobiliarios        0
qtd_passou_60_89_dias       0
qtd_dependentes             0
grupo_idade                 0
classe_financeira           0
dtype: int64

In [512]:
x_test.isna().sum()

inadimplente                0
linhas_inseguras            0
idade                       0
qtd_passou_de_30_59_dias    0
razao_debito                0
salario_mensal              0
qtd_crdto_aberto            0
qtd_passou_90_dias          0
qtd_emp_imobiliarios        0
qtd_passou_60_89_dias       0
qtd_dependentes             0
grupo_idade                 0
classe_financeira           0
dtype: int64

In [513]:
# model
lr = LinearRegression().fit( x_train, y_train )

# prediction
yhat_lr = lr.predict( x_test )

# performance
lr_result = ml_error( 'Linear Regression', y_test, yhat_lr )
lr_result



,Model Name,MAE,MAPE,RMSE
0,Linear Regression,3.588262e-16,inf,9.434389e-16


## 7.3. Random Forest Regressor

In [514]:
# model
rf = RandomForestRegressor( n_estimators=100, n_jobs=-1, random_state=42 ).fit( x_train, y_train )

# prediction
yhat_rf = rf.predict( x_test )

# performance
rf_result = ml_error( 'Random Forest Regressor', np.expm1( y_test ), np.expm1( yhat_rf ) )
rf_result

,Model Name,MAE,MAPE,RMSE
0,Random Forest Regressor,0.0,NaN,0.0


### 7.4. XGBoost Regressor

In [515]:


# model
model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                              n_estimators=100, 
                              eta=0.01, 
                              max_depth=10, 
                              subsample=0.7,
                              colsample_bytee=0.9 ).fit( x_train, y_train )

# prediction
yhat_xgb = model_xgb.predict( x_train )

# performance
xgb_result = ml_error( 'XGBoost Regressor', np.expm1( y_test ), np.expm1( yhat_xgb ) )
xgb_result



[02:37:28] WARNING: ../src/learner.cc:576: 
Parameters: { "colsample_bytee" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




ValueError: Found input variables with inconsistent numbers of samples: [40000, 110000]